In [1]:
import pynbody
import pandas as pd
import numpy as np
import pickle

from base import *
from analysis import *
from compiler_test import *

import tqdm

In [ ]:
keys = ['h148_13','h148_28','h148_37','h148_45','h148_68','h148_80','h148_283',
        'h148_278','h148_329','h229_20','h229_22','h229_23','h229_27','h229_55',
        'h242_24','h242_41','h242_80','h329_33','h329_137']

In [2]:
sim = 'h242'
haloid= 41
verbose = False



key = f'{sim}_{str(int(haloid))}'

path = f'{rootPath}Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
discharged = pd.read_hdf(path, key=key)

path = f'{rootPath}Stellar_Feedback_Code/SNeData/dsrg_accreted_particles.hdf5'
accreted = pd.read_hdf(path, key=key)


if verbose: print(f'Now computing reaccreted particles for {sim}-{haloid}...')
reaccreted = pd.DataFrame() # gas accreted following a discharge event.

pids = np.unique(discharged.pid) # quicker to use 'discharged' because fewer unique particles.

# defining attribute giving the length of time between discharge and accretion event for each gas particle:
recycleTime = {'recycleTime': ""} 
accreted = accreted.join(pd.DataFrame(columns=recycleTime)) 
discharged = discharged.join(pd.DataFrame(columns=recycleTime))
# ensuring that our new accreted dataframe inherits sne heating identified 'hot'.
hot = {'hot':  ""}
accreted = accreted.join(pd.DataFrame(columns=hot))
discharged = discharged.join(pd.DataFrame(columns=hot))    

for pid in tqdm.tqdm(pids):
    dis = discharged[discharged.pid==pid]
    acc = accreted[accreted.pid==pid]

    dTime = np.asarray(dis.time)
    aTime = np.asarray(acc.time)


    # removing initial accretion event if it does not correspond to a discharge; else no alterations.
    # check to ensure that our discharge event actually has an accretion:
    if (len(dTime) == 0):
        print('Problem: No instance of discharge per particle for '+str(pid))
        continue
    if (len(aTime) == 0): # if no instances of accretion, move on to next particle
        if (len(dis) != 1):
            print('Problem: multiple discharges but no accretion events for '+str(pid))
            continue
        aCache = dis[0]
        aCache['recycleTime'] = np.array([-1])
        reaccreted = pd.concat([reaccreted, aCache])
        continue
    if (aTime[0] < dTime[0]): # In there is an initial accretion
        aCache = acc[1:]
    else:
        aCache = acc

    dCache = dis #[0:len(aCache)]

    aCache['recycleTime'] = np.array(aCache['time']) - (np.array(dCache['time'])[0:len(aCache)])
#         print(dCache['time'],aCache['time'], aCache['recycleTime'])
    aCache['hot'] = dCache['hot']
    if len(dCache) != len(aCache):
        dCache.iloc[-1]['recycleTime'] = -1
        aCache = aCache.append(dCache.iloc[-1])


    reaccreted = pd.concat([reaccreted, aCache])

  0%|          | 0/1084 [00:00<?, ?it/s]


KeyError: 0

In [2]:
sim = 'h242'
haloid= 41
verbose = False

import tqdm
data = read_tracked_particles(sim, haloid, verbose=verbose)


if verbose: print(f'Now computing discharged particles for {sim}-{haloid}...')
predischarged = pd.DataFrame() # discharged gas particles but with their properties before discharge.
discharged = pd.DataFrame() # gas particles that are removed from their satellite's disk
dsrg_accreted = pd.DataFrame() # gas accreted following a discharge event.


pids = np.unique(data.pid)
for pid in tqdm.tqdm(pids):
    dat = data[data.pid==pid]

    sat_disk = np.array(dat.sat_disk, dtype=bool)
    in_sat = np.array(dat.in_sat, dtype=bool)
    outside_disk = ~sat_disk

    time = np.array(dat.time, dtype=float)

    for i,t2 in enumerate(time[1:]):
            i += 1
            if (sat_disk[i-1] and outside_disk[i]):
                in_ = dat[time==time[i-1]].copy()
                out = dat[time==t2].copy()
                predischarged = pd.concat([predischarged, in_])
                discharged = pd.concat([discharged, out])


#             # specifically picking out that gas accreted after one time step.
#             if (outside_disk[i-1] and sat_disk[i]):
#                 acc = dat[time==t2].copy()
#                 dsrg_accreted = pd.concat([dsrg_accreted, acc])


# # apply the calc_angles function along the rows of discharged particles.
# print('Calculating predischarge angles.')
# predischarged = predischarged.apply(calc_angles, axis=1)
# print('Calculating discharged angles.')
# discharged = discharged.apply(calc_angles, axis=1)


# adding a key to tell us whether or not the particle underwent supernova heating just prior to discharge.    
# hot = {'hot': ""} 
# discharged = discharged.join(pd.DataFrame(columns=hot))

print('Appending `heating` boolean to discharged.')

coolontime = np.asarray(discharged.coolontime)
time = np.asarray(predischarged.time)

discharged['heating'] = coolontime > time

100%|██████████| 5672/5672 [00:14<00:00, 384.52it/s]

Appending `heating` boolean to discharged.


In [35]:
discharged


,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,dt,r_gal,host_r_gal,sat_disk,sat_halo,host_disk,host_halo,other_sat,IGM,heating
13,9.383249,3122411,0.027334,11880.298828,26512.469228,0.000000,1.179405,0.029980,0.330287,-0.601129,...,0.323235,0.346348,19.193015,False,True,False,False,False,False,False
28,9.383249,3427663,0.044766,11170.382812,27154.510990,8.998238,0.583680,0.014837,-0.278413,-0.494204,...,0.323235,0.346348,19.193015,False,True,False,False,False,False,False
28,10.029719,3427663,0.007828,13244.921875,27305.883390,10.021814,1.170723,0.028177,-0.375370,1.091348,...,0.323235,0.358063,14.143859,False,True,False,False,False,False,True
34,11.645896,3427915,0.044088,10415.166992,26574.890266,0.000000,0.867387,0.018513,0.223383,-0.822962,...,0.323235,0.357445,16.893003,False,True,False,False,False,False,False
35,10.352954,3427916,0.001827,16058.614258,26656.824788,0.000000,3.223554,0.075444,2.128210,2.078896,...,0.323235,0.364196,14.430474,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5668,10.767100,5549088,0.037385,10829.878906,27355.842327,10.706162,1.408204,0.031810,-0.118166,-1.232752,...,0.090910,0.354976,16.023003,False,True,False,False,False,False,True
5669,9.706484,5553452,0.030194,11261.113281,26787.996356,9.480841,1.083100,0.026850,-0.587699,0.756881,...,0.323235,0.348828,14.573041,False,True,False,False,False,False,True
5669,10.676190,5553452,0.002312,18633.841797,26807.306051,9.480841,2.766462,0.062960,1.542851,-2.133397,...,0.323235,0.326757,12.563484,False,True,False,False,False,False,False
5669,11.969131,5553452,0.008693,12252.198242,26821.243020,9.480841,1.286725,0.027757,0.610865,0.829337,...,0.323235,0.372926,15.294217,False,True,False,False,False,False,False


In [33]:
discharged[['coolontime', 'time','hot']]

,coolontime,time,hot
13,0.000000,9.383249,False
28,8.998238,9.383249,False
28,10.021814,10.029719,False
34,0.000000,11.645896,False
35,0.000000,10.352954,False
...,...,...,...
5668,10.706162,10.767100,False
5669,9.480841,9.706484,False
5669,9.480841,10.676190,False
5669,9.480841,11.969131,False


In [37]:
dat = discharged.coolontime > discharged.time

In [21]:
i=0
for i, bl in enumerate(np.array(dat)):
    if bl == True:
        print(bl)
        print(i)
        

True
15
True
34
True
45
True
58
True
78
True
124
True
177
True
236
True
312
True
405
True
478
True
521
True
593
True
653
True
685
True
693
True
814
True
975
True
983
True
1016
True
1257
True
1335
True
1377
True
1498
True
1560
True
1590
True
1597
True
1657
True
1700


In [16]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
sim = 'h242'
haloid= 41
verbose = False





import tqdm
key = f'{sim}_{str(int(haloid))}'

# path = f'{rootPath}Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
# discharged = pd.read_hdf(path, key=key)

path = f'{rootPath}Stellar_Feedback_Code/SNeData/dsrg_accreted_particles.hdf5'
accreted = pd.read_hdf(path, key=key)


if verbose: print(f'Now computing adv. accreted particles for {sim}-{haloid}...')
reaccreted = pd.DataFrame() # gas accreted following a discharge event.




# defining attribute giving the length of time between discharge and accretion event for each gas particle:
recycleTime = {'recycleTime': ""} 
accreted = accreted.join(pd.DataFrame(columns=recycleTime)) 
# # ensuring that our new accreted dataframe inherits sne heating identified 'hot'.
heating = {'heating': ''} 
accreted = accreted.join(pd.DataFrame(columns=heating))      



pids = np.unique(discharged.pid) # quicker to use 'discharged' because fewer unique particles.
for pid in tqdm.tqdm(pids):
    dis = discharged[discharged.pid==pid]
    acc = accreted[accreted.pid==pid]

    dTime = np.asarray(dis.time)
    aTime = np.asarray(acc.time)


    # removing initial accretion event if it does not correspond to a discharge; else no alterations.
    # check to ensure that our discharge event actually has an accretion:
    if (len(aTime) == 0) or (len(dTime) == 0): # if no instances of accretion or none of discharge, move on to next particle.
        continue
    if (aTime[0] < dTime[0]):
        aCache = acc[1:]

    else:
        aCache = acc

    if len(aCache) == 0: # if no instances of reaccretion, move on to next particle.
        continue

        
    dCache = dis[0:len(aCache)]
    aCache['recycleTime'] = np.array(aCache['time']) - np.array(dCache['time'])
    
    heating = np.array(dCache['heating'])
    aCache['heating'] = heating
    reaccreted = pd.concat([reaccreted, aCache])
    


100%|██████████| 1084/1084 [00:02<00:00, 369.74it/s]


In [20]:
reaccreted

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,sat_disk,sat_halo,host_disk,host_halo,other_sat,IGM,t,sat_Mvir,recycleTime,heating
11372,9.706484,3427663,2.564758,1213.669800,27217.735088,8.998238,0.319184,0.007913,-0.250604,0.193689,...,True,False,False,False,False,False,9.706484,5.474459e+09,0.323235,False
22727,10.352954,3427979,0.276933,9536.864258,26558.376600,0.000000,0.370127,0.008662,-0.000073,-0.341193,...,True,False,False,False,False,False,10.352954,5.544178e+09,0.323235,False
56759,11.969131,3427979,0.548262,8720.259766,26563.390109,0.000000,0.216194,0.004664,0.200504,-0.030592,...,True,False,False,False,False,False,11.969131,4.863942e+09,1.292941,False
28407,10.676190,3428006,1.115964,4220.712891,26693.116170,9.870848,0.292932,0.006667,-0.198061,-0.117743,...,True,False,False,False,False,False,10.676190,5.577673e+09,0.323235,False
39754,10.999425,3428012,0.290898,9354.319336,26558.323062,0.000000,0.367634,0.008155,-0.126284,0.334509,...,True,False,False,False,False,False,10.999425,5.583368e+09,0.969706,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45344,10.999425,4000556,1.146601,3904.018555,26691.525594,9.956999,0.265760,0.005895,0.140649,-0.186307,...,True,False,False,False,False,False,10.999425,5.583368e+09,0.232325,False
68038,12.150951,4000569,1.081567,7003.306152,26620.642208,8.707832,0.278619,0.006296,0.119458,-0.218818,...,True,False,False,False,False,False,12.150951,4.194303e+09,1.383851,False
22685,10.029719,5553452,0.503866,9266.349609,26803.420279,9.480841,0.270961,0.006521,0.204157,0.169048,...,True,False,False,False,False,False,10.029719,5.516836e+09,0.323235,True
56717,11.645896,5553452,0.182225,9468.704102,26819.901133,9.480841,0.485823,0.010369,-0.378405,0.179700,...,True,False,False,False,False,False,11.645896,5.402405e+09,0.969706,False
